In [1]:
# from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

# llm = OpenAI(model_name="gpt-3.5-turbo-1106") // 이게 사용료가 더 비쌈
# chat = ChatOpenAI(model_name="gpt-4") # 기본은 gpt-3.5-turbo-0301
chat = ChatOpenAI(
    temperature=0.1     # 숫자가 높을수록 랜덤함.
)

#a = llm.predict("How many planets are there?")
#b = chat.predict("How many planets are there?")

#b

In [2]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="You are a geography expert. And you only reply in Italian."),
    AIMessage(content="Ciao, mi chiamo Paolo!"),
    HumanMessage(content="What is the distance between Mexico and Thailand. Also, what is your name?")
]

chat.predict_messages(messages)

AIMessage(content='La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri. Come posso aiutarti oggi?')